![](https://wherobots.com/wp-content/uploads/2023/12/Inline-Blue_Black_onWhite.png)
</br>
<div style="display: flex;justify-content: flex-end; align-items: center; max-width: 100%; margin: auto; gap: 90px;">
    <!-- Main Content Box -->
    <div style="position: relative; display: flex; flex-direction: column; gap: 30px; 
                font-family: Arial, sans-serif; line-height: 1.6; max-width: 800px; 
                padding: 30px; color: #ffffff; background: #121212; 
                border-radius: 10px; box-shadow: 0px 0px 20px rgba(0, 255, 255, 0.0); 
                overflow: hidden;">
        <!-- Introductory Text -->
        <div style="position: relative; z-index: 2;">
               <h2 style="color: #00f5d4;">WherobotsAI Raster Inference - Object Detection</h2>
            <p style="font-size: 18px; color: #e0e0e0;">
                This example demonstrates an object detection model 
                with <strong>Raster Inference</strong> to identify <span style="color: #00f5d4; font-weight: bold;">
                marine infrastructure</span> (offshore wind farms and platforms) in satellite imagery.
            </p>
            <p style="font-size: 18px; color: #e0e0e0;">
                We will use a <strong>machine-learning model</strong> from <span style="color: #ff007f; font-weight: bold;">
                Satlas</span>, which was trained using imagery from the 
                <strong>European Space Agency’s Sentinel-2 satellites</strong>.
            </p>
        </div>
    </div>
    <!-- Right Section: Icons -->
    <div style="display: flex; flex-direction: column; align-items: flex-end; gap: 20px">
        <img src="assets/offshore_oil.png" 
             alt="Setup Icon" 
             style="width: 100%; height: 100%; border-radius: 8px;margin: 20px">
        <img src="assets/wind_farm.png" 
             alt="Wind Engine Offshore" 
             style="width: 100%; height: 100%; border-radius: 8px;margin: 20px ">
    </div>

</div>

</br>


</br>
<div style="position: relative; display: flex; flex-direction: column; gap: 20px; 
            font-family: Arial, sans-serif; line-height: 1.6; max-width: 90%; 
            margin: auto; padding: 30px; color: #ffffff; background: #121212; 
            border-radius: 10px;); 
            overflow: hidden;">
    <!-- Title Section -->
    <div style="position: relative; z-index: 2; text-align: left;">
        <h2 style="color: #00f5d4;">Set Up The WherobotsDB Context</h2>
    </div>
    <!-- Main Content -->
    <div style="position: relative; z-index: 2;">
        <p style="font-size: 18px; color: #e0e0e0;">
            Here we configure WherobotsDB to enable access to the necessary cloud object storage buckets with sample data.
        </p>
    </div>
    <!-- Right Section: AI Icon -->
    <div style="display: flex; justify-content: flex-end; position: relative; z-index: 2; margin-top: 10px;">
    </div>
</div>
</br>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from sedona.spark import SedonaContext
from pyspark.sql.functions import expr, size, col
from sedona.maps.SedonaKepler import SedonaKepler
from sedona.raster_utils.SedonaUtils import SedonaUtils
import json

config = SedonaContext.builder().appName('object-detection-batch-inference')\
    .getOrCreate()

sedona = SedonaContext.create(config)

</br>
<div style="position: relative; display: flex; flex-direction: column; gap: 20px; 
            font-family: Arial, sans-serif; line-height: 1.6; max-width: 90%; 
            margin: auto; padding: 30px; color: #ffffff; background: #121212; 
            border-radius: 10px;); 
            overflow: hidden;">
    <!-- Title Section -->
    <div style="position: relative; z-index: 2; text-align: left;">
        <h2 style="color: #00f5d4;">Load Satellite Imagery Efficiently</h2>
    </div>
    <!-- Main Content -->
    <div style="position: relative; z-index: 2;">
<p style="font-size: 18px; color: #e0e0e0;">
            In this step, we load the satellite imagery to run <strong style="color: #00f5d4;">inference</strong> over.  
            These <strong>GeoTIFF images</strong> are ingested as <strong style="color: #00f5d4;">out-of-database or "out-db" rasters</strong> in 
            <strong>WherobotsDB</strong> and stored in the Spatial Catalog for easy access. Building out DB ensuring efficient storage and retrieval. You can learn more about his process here.
        </p>
    </div>
    <!-- Right Section: AI Icon -->
    <div style="display: flex; justify-content: flex-end; position: relative; z-index: 2; margin-top: 10px;">
    </div>
</div>
</br>

In [ ]:
df_raster_input = sedona.table(f"wherobots_pro_data.satlas.offshore_satlas")

In [ ]:
df_raster_input.printSchema()

In [ ]:
df_raster_input.count()

</br>
<div style="position: relative; display: flex; flex-direction: column; gap: 20px; 
            font-family: Arial, sans-serif; line-height: 1.6; max-width: 90%; 
            margin: auto; padding: 30px; color: #ffffff; background: #121212; 
            border-radius: 10px;); 
            overflow: hidden;">
    <!-- Title Section -->
    <div style="position: relative; z-index: 2; text-align: left;">
        <h2 style="color: #00f5d4;">Focus on a Coastal Region</h2>
    </div>
    <!-- Main Content -->
    <div style="position: relative; z-index: 2;">
        <p style="font-size: 18px; color: #e0e0e0;">
            With <strong>176,000 images</strong> covering most of Earth's coastlines, let's choose an area to focus on.
        </p>
        <p style="font-size: 18px; color: #e0e0e0;">
            Draw a <span style="color: #00f5d4; font-weight: bold;">polygon</span> around the <strong>Yellow Sea</strong> 
            off the east coast of China to define an area of interest (AOI).
        </p>
    </div>
    <!-- Right Section: AI Icon -->
    <div style="display: flex; justify-content: flex-end; position: relative; z-index: 2; margin-top: 10px;">
    </div>
</div>
</br>


In [ ]:
from leafmap import Map

my_map = Map()
my_map

In [ ]:
my_map.user_roi # this shows the last draw feature

In [ ]:
feature_json = json.dumps(my_map.user_roi) # formats the python dictionary as a string so we can pass it to SQL
df_raster_sub = df_raster_input.where(
    expr(f"""ST_INTERSECTS(footprint, ST_GeomFromGeoJSON('{feature_json}'))""")
)

df_raster_sub.cache()
print(f"IMAGE COUNT: {df_raster_sub.count()}")
df_raster_sub.show(3, truncate=True)
df_raster_sub.createOrReplaceTempView("df_raster_input")

</br>
<div style="position: relative; display: flex; flex-direction: column; gap: 20px; 
            font-family: Arial, sans-serif; line-height: 1.6; max-width: 90%; 
            margin: auto; padding: 30px; color: #ffffff; background: #121212; 
            border-radius: 10px; 
            overflow: hidden;">
    <!-- Main Content -->
    <div style="position: relative; z-index: 2;text-align: left;">
        <h2 style="color: #00f5d4;text-align: left;">Viewing Results</h2>
        <p style="font-size: 18px; color: #e0e0e0;">
           With our AOI defined we can see the footprints of the images in the area with the <code>SedonaKepler.create_map()</code> integration .
        </p>
        <p style="font-size: 18px; color: #e0e0e0;">
           Using <code>SedonaUtils.display_image()</code> we can view the images as well.
        </p>
    </div>
    </br>
    <!-- Right Section: Map Icon -->
    <div style="display: flex; justify-content: flex-end; position: relative; z-index: 2; margin-top: 10px;">

</div>
</br>

In [ ]:
from sedona.maps.SedonaKepler import SedonaKepler

map = SedonaKepler.create_map()

SedonaKepler.add_df(map, df=df_raster_sub, name="Image Footprints")

map

In [ ]:
%%time
htmlDf = sedona.sql(f"""SELECT RS_AsImage(outdb_raster, 250), name as FROM df_raster_input limit 10""")
SedonaUtils.display_image(htmlDf)

</br>
<div style="position: relative; display: flex; flex-direction: column; gap: 20px; 
            font-family: Arial, sans-serif; line-height: 1.6; max-width: 90%; 
            margin: auto; padding: 30px; color: #ffffff; background: #121212; 
            border-radius: 10px;); 
            overflow: hidden;">
    <!-- Title Section -->
    <div style="position: relative; z-index: 2; text-align: left;">
        <h2 style="color: #00f5d4;">Run Predictions And Visualize Results</h2>
    </div>
    <!-- Main Content -->
    <div style="position: relative; z-index: 2;">
        <p style="font-size: 18px; color: #e0e0e0;">
            To run predictions, we will specify the model we wish to use. Some models are pre-loaded and made available in 
            <strong>Wherobots Cloud</strong>. We can also load our own models. 
        </p>
        <p style="font-size: 18px; color: #e0e0e0;">
            Inference can be run using <strong>Wherobots' Spatial SQL functions</strong>, in this case: <code>RS_DETECT_BBOXES()</code>
        </p>
        <p style="font-size: 18px; color: #e0e0e0;">
            Here we generate predictions for the all images in the AOI.
        </p>
    </div>
    <!-- Right Section: AI Icon -->
    <div style="display: flex; justify-content: flex-end; position: relative; z-index: 2; margin-top: 10px;">
    </div>
</div>
</br>

In [ ]:
model_id = 'marine-satlas-sentinel2'

predictions_df = sedona.sql(f"""
SELECT
  outdb_raster,
  name as image_name,
  detect_result.*
FROM (
  SELECT
    outdb_raster,
    name,
    RS_DETECT_BBOXES('{model_id}', outdb_raster) AS detect_result
  FROM
    df_raster_input
) AS detect_fields
""")

predictions_df.cache().count()
predictions_df.filter(size(col("labels")) == 0).show(3)
predictions_df.where('image_name = "449623202-8-8.tiff"').show()
predictions_df.createOrReplaceTempView("predictions")



</br>
<div style="position: relative; display: flex; flex-direction: column; gap: 20px; 
            font-family: Arial, sans-serif; line-height: 1.6; max-width: 90%; 
            margin: auto; padding: 30px; color: #ffffff; background: #121212; 
            border-radius: 10px;); 
            overflow: hidden;">
    <!-- Title Section -->
    <div style="position: relative; z-index: 2; text-align: left;">
        <h2 style="color: #00f5d4;">Run Predictions And Visualize Results</h2>
    </div>
    <!-- Main Content -->
    <div style="position: relative; z-index: 2;">
        <p style="font-size: 18px; color: #e0e0e0;">
            Since we ran inference across many country coastlines all over the world, many scenes don't contain wind farms and don't have positive detections. Now that we've generated predictions using our model over our satellite imagery, we can filter the geometries by confidence score with <code>RS_FILTER_BOX_CONFIDENCE</code> and by the integer label representing offshore wind farms, <code>1</code>, to locate predicted offshore wind farms.
        </p>
    </div>
    <!-- Right Section: AI Icon -->
    <div style="display: flex; justify-content: flex-end; position: relative; z-index: 2; margin-top: 10px;">
    </div>
</div>
</br>

In [ ]:
filtered_predictions = sedona.sql(f"""
  SELECT
    outdb_raster,
    image_name,
    filtered.*
  FROM (
    SELECT
      outdb_raster,
      image_name,
      RS_FILTER_BOX_CONFIDENCE(bboxes_wkt, confidence_scores, labels, 0.65) AS filtered
    FROM
      predictions
  ) AS temp
    WHERE size(filtered.max_confidence_bboxes) > 0
    AND array_contains(filtered.max_confidence_labels, '1')
""")
filtered_predictions.createOrReplaceTempView("filtered_predictions")


</br>
<div style="position: relative; display: flex; flex-direction: column; gap: 20px; 
            font-family: Arial, sans-serif; line-height: 1.6; max-width: 90%; 
            margin: auto; padding: 30px; color: #ffffff; background: #121212; 
            border-radius: 10px;); 
            overflow: hidden;">
    <!-- Title Section -->
    <div style="position: relative; z-index: 2; text-align: left;">
        <h2 style="color: #00f5d4;">Prepare Results</h2>
    </div>
    <!-- Main Content -->
    <div style="position: relative; z-index: 2;">
        <p style="font-size: 18px; color: #e0e0e0;">
            Our final step before plotting our prediction results is to convert our table from a format where each row represents a raster scene's predictions to a format where each row represents one predicted bounding box. To do this, we combine our list columns with <code>arrays_zip</code> and then use <code>explode</code> to convert lists to rows. To convert our string column representing a geometry into a <code>GeometryType</code> column, we use <code>ST_GeomFromWKT</code> so we can plot it with <code>SedonaKepler</code>.
        </p>
    </div>
    <!-- Right Section: AI Icon -->
    <div style="display: flex; justify-content: flex-end; position: relative; z-index: 2; margin-top: 10px;">
    </div>
</div>
</br>

In [ ]:
exploded_df = sedona.sql("""
SELECT
    outdb_raster,
    image_name,
    exploded.*
FROM (
    SELECT
        outdb_raster,
        image_name,
        explode(arrays_zip(max_confidence_bboxes, max_confidence_scores, max_confidence_labels)) AS exploded
    FROM
        filtered_predictions
) temp
""")
df_exploded = exploded_df.withColumn("geometry", expr("ST_GeomFromWkt(max_confidence_bboxes)")).drop("max_confidence_bboxes")
print(df_exploded.cache().count())
df_exploded.show()

In [ ]:
from sedona.maps.SedonaKepler import SedonaKepler

map = SedonaKepler.create_map()

SedonaKepler.add_df(map, df=df_exploded.drop("outdb_raster"), name="Wind Farm Detections")
map


</br>
<div style="position: relative; display: flex; flex-direction: column; gap: 20px; 
            font-family: Arial, sans-serif; line-height: 1.6; max-width: 90%; 
            margin: auto; padding: 30px; color: #ffffff; background: #121212; 
            border-radius: 10px;); 
            overflow: hidden;">
    <!-- Title Section -->
    <div style="position: relative; z-index: 2; text-align: left;">
        <h2 style="color: #00f5d4;">Select a Footprint and Review the Image</h2>
    </div>
    <!-- Main Content -->
    <div style="position: relative; z-index: 2;">
        <p style="font-size: 18px; color: #e0e0e0;">
        Select one of the detected <strong>footprints</strong> from the map above. 
        Copy the name of a detected bounding box and paste it into the query below 
        to retrieve the corresponding image.
        </p>
    </div>
    <!-- Right Section: AI Icon -->
    <div style="display: flex; justify-content: flex-end; position: relative; z-index: 2; margin-top: 10px;">
    </div>
</div>
</br>


In [ ]:
%%time
image_name = '2015411785-6-6.tiff'
htmlDf = sedona.sql(f"""SELECT RS_AsImage(outdb_raster, 500), name as FROM df_raster_input WHERE name = '{image_name}' """)
SedonaUtils.display_image(htmlDf)

### wherobots.inference Python API

If you prefer python, wherobots.inference offers a module for registering the SQL inference functions as python functions. Below we run the same inference as before with `RS_DETECT_BBOXES`.

In [ ]:
from wherobots.inference.engine.register import create_object_detection_udfs
from pyspark.sql.functions import col
rs_detect, rs_threshold_geoms =  create_object_detection_udfs(batch_size = 10, sedona=sedona)
df = df_raster_input.withColumn("detect_result", rs_detect(model_id, col("outdb_raster"))).select(
                               "outdb_raster",
                               col("detect_result.bboxes_wkt").alias("bboxes_wkt"),
                               col("detect_result.confidence_scores").alias("confidence_scores"),
                               col("detect_result.labels").alias("labels")
                           )
df.show()